In [12]:
def func(X,R):
        """
        # calculating Y
        Y = R*X

        #Decomposing matrix
        U, s, V = np.linalg.svd(X.T, full_matrices = False)
        s = np.diag(s)

        #print U.shape, s.shape
        U = np.dot(U, s.T)
        #print U.shape, V.shape
        V = V.T

        #X_0 = (np.dot(U, V)).T
        X_0 = np.dot(V,U)
        #print X_0.shape
        def soft(t,s):
            x = np.sign(t)
            print x
            z = np.linalg.det(t)
            print z
            x = x*np.maximum(0, z-s)
            print x
            return x

        for i in range(10):
            X_0 = np.dot(V,U)
            Z = X_0 + (Y - R*X_0)
            V = np.dot((np.dot(Z, U.T)), (np.linalg.inv(np.dot(U,U.T) + lamda*np.eye(n_users))))
            X_0 = np.dot(V,U)
            W = X_0 + R* (Y - R*X_0) 
            alpha = 1.01 * np.max(np.linalg.eig(np.dot(V.T, V))[0])
            print alpha
            U = soft(U + (1/alpha)*(np.dot(V.T, (W - np.dot(V,U)))), lamda/(2*alpha))
            print U
            U = np.dot((np.linalg.inv(np.dot(V.T,V))),(np.dot(V.T, Z))) + lamda*np.sign(np.linalg.norm(U))

        X_0 = np.trunc(np.dot(V,U))

        for i in range(len(X_0)):
            for j in range(len(X_0[0])):
                X_0[i][j] = np.abs(X_0[i][j])
        print X_0
        return X_0, V
        """
        U, s, V = np.linalg.svd(X, full_matrices=True)
        #print U.shape, s.shape, V.shape
        s = np.diag(s)
        #print s
        d = 70
        RL = np.dot(np.dot(U[:,1:d], s[1:d, 1:d]), V[:, 1:d].T)
        return RL



In [33]:
def kfold_100k(fold):
    loc_train = "ml-100k/u" + str(fold) + ".base"
    loc_test = "ml-100k/u" + str(fold) + ".test"
    header = ['user_id', 'item_id', 'rating', 'timestamp']
    train_data = pd.read_csv(loc_train, sep = '\t', names = header)
    test_data = pd.read_csv(loc_test, sep = '\t', names = header)
    
    rmin = 1
    rmax = 5
    R = np.zeros((n_users,n_items))
    
    #user-item matrix, train and test matrices
    train_matrix = np.zeros((n_users, n_items))
    for record in train_data.itertuples():
        train_matrix[record[1] -1][record[2] -1] = record[3]
        
    X = train_matrix
        
    k = int(0.25*(1682))
    #dividing matrix in 4 parts
    C1 = X[:,:k]
    C2 = X[:,k+1:2*k]
    C3 = X[:,2*k+1:3*k]
    C4 = X[:,3*k:]
    
    
    test_matrix = np.zeros((n_users, n_items))
    for record in test_data.itertuples():
        test_matrix[record[1] -1][record[2] -1] = record[3]
        
    test_matrix = test_matrix
    
    #Mask matrix R
    indices = np.nonzero(X)
    R[indices] = 1
    R1 = R[:,:k]
    R2 = R[:,k+1:2*k]
    R3 = R[:,2*k+1:3*k]
    R4 = R[:,3*k:]
    
    C_1 = func(C1,R1)
    C_2 = func(C2,R2)
    C_3 = func(C3,R3)
    C_4 = func(C4,R4)
    
    C1_prod = np.dot(np.dot(C_1,np.linalg.inv(np.dot(C_1.T,C_1))),C_1.T)
    L12 = np.dot(C1_prod,C_2)
    L13 = np.dot(C1_prod,C_3)
    L14 = np.dot(C1_prod,C_4)
    
    C2_prod = np.dot(np.dot(C_2,np.linalg.inv(np.dot(C_2.T,C_2))),C_2.T)
    L21 = np.dot(C2_prod,C_1)
    L23 = np.dot(C2_prod,C_3)
    L24 = np.dot(C2_prod,C_4)
    
    C3_prod = np.dot(np.dot(C_3,np.linalg.inv(np.dot(C_3.T,C_3))),C_3.T)
    L31 = np.dot(C3_prod,C_1)
    L32 = np.dot(C3_prod,C_2)
    L34 = np.dot(C3_prod,C_4)
    
    C4_prod = np.dot(np.dot(C_4,np.linalg.inv(np.dot(C_4.T,C_4))),C_4.T)
    L41 = np.dot(C4_prod,C_1)
    L42 = np.dot(C4_prod,C_2)
    L43 = np.dot(C4_prod,C_3)
    
    #L1 = np.mean(C_1,L21,L31,L41)
    #L2 = np.mean(L12,C_2,L32,L42)
    #L3 = np.mean(L13,L23,C_3,L43)
    #L4 = np.mean(L14,L24,L34,C_4)
    
    
    
    predicted_new = np.column_stack((C_1,L12,L13,L14))
    
    predicted = np.column_stack((C_1,C_2,C_3,C_4))
    print predicted.shape

    from sklearn.metrics import mean_absolute_error as mae

    def nmae(prediction, ground_truth):
        #print prediction[ground_truth.nonzero()].flatten()
        prediction = prediction[ground_truth.nonzero()].flatten()
        ground_truth = ground_truth[ground_truth.nonzero()].flatten()
        error = mae(ground_truth, prediction)
        #print error
        return error/(rmax-rmin)

    print "NMAE for Fold " + str(fold) + " : " +  str(nmae(predicted, test_matrix))
    print "NMAE new for Fold " + str(fold) + " : " +  str(nmae(predicted_new, test_matrix))


In [34]:
import numpy as np
import pandas as pd


n_users = 943
n_items = 1682

lamda = 1

for lamda in [0.001, 0.005, 0.05, 1]:    
    for i in range(1, 2):
            print '\nResults for Fold ' + str(i) + ' :'
            kfold_100k(i)


Results for Fold 1 :
(943, 1680)
NMAE for Fold 1 : 0.888372474801
NMAE new for Fold 1 : 5.52725866664

Results for Fold 1 :
(943, 1680)
NMAE for Fold 1 : 0.888372474801
NMAE new for Fold 1 : 5.52725866664

Results for Fold 1 :
(943, 1680)
NMAE for Fold 1 : 0.888372474801
NMAE new for Fold 1 : 5.52725866664

Results for Fold 1 :
(943, 1680)
NMAE for Fold 1 : 0.888372474801
NMAE new for Fold 1 : 5.52725866664
